# Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Dropout, GlobalAveragePooling1D, Dense
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import gc

In [2]:
nucleo_dic = {
    "A": 0,
    "R": 1,
    "N": 2,
    "D": 3,
    "C": 4,
    "Q": 5,
    "E": 6,
    "G": 7,
    "H": 8,
    "I": 9,
    "L": 10,
    "K": 11,
    "M": 12,
    "F": 13,
    "P": 14,
    "S": 15,
    "T": 16,
    "W": 17,
    "Y": 18,
    "V": 19,
    "X": 20
}


In [3]:
!git lfs clone https://huggingface.co/EvaKlimentova/knots_simple_CNN

          with new flags from 'git clone'

'git clone' has been updated in upstream Git to have comparable
speeds to 'git lfs clone'.
Cloning into 'knots_simple_CNN'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Git LFS: (1 of 1 files) 187.61 KB / 187.61 KB


In [4]:
model = tf.keras.models.load_model('knots_simple_CNN/cnn_10epochs.h5')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 493, 32)           5408      
                                                                 
 batch_normalization_3 (Batc  (None, 493, 32)          128       
 hNormalization)                                                 
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 246, 32)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 239, 16)           4112      
                                                                 
 batch_normalization_4 (Batc  (None, 239, 16)          64        
 hNormalization)                                                 
                                                      

## Integrated Gradients

In [5]:
def generate_alphas(m_steps=50, method='riemann_trapezoidal'):
    """
    Args:
    m_steps(Tensor): A 0D tensor of an int corresponding to the number of linear
    interpolation steps for computing an approximate integral. Default is 50.
    method(str): A string representing the integral approximation method. The
       following methods are implemented:
      - riemann_trapezoidal(default)
      - riemann_left
      - riemann_midpoint
      - riemann_right
    Returns:
      alphas(Tensor): A 1D tensor of uniformly spaced floats with the shape
      (m_steps,).
      """
    m_steps_float = tf.cast(m_steps, float)

    if method == 'riemann_trapezoidal':
        alphas = tf.linspace(0.0, 1.0, m_steps+1)
    elif method == 'riemann_left':
        alphas = tf.linspace(0.0, 1.0 - (1.0 / m_steps_float), m_steps)
    elif method == 'riemann_midpoint':
        alphas = tf.linspace(1.0 / (2.0 * m_steps_float), 1.0 - 1.0 / (2.0 * m_steps_float), m_steps)
    elif method == 'riemann_right':
        alphas = tf.linspace(1.0 / m_steps_float, 1.0, m_steps)
    else:
        raise AssertionError("Provided Riemann approximation method is not valid.")

    return alphas

def generate_path_inputs(baseline, input, alphas):
    """
    Generate interpolated 'images' along a linear path at alpha intervals between a baseline tensor

    baseline: 2D, shape: (500, 21)
    input: preprocessed sample, shape: (500, 21)
    alphas: list of steps in interpolated image ,shape: (21)


    return: shape (21, 500, 21)
    """
    # Expand dimensions for vectorized computation of interpolations.
    alphas_x = alphas[:, tf.newaxis, tf.newaxis]
    baseline_x = tf.expand_dims(baseline, axis=0)
    input_x = tf.expand_dims(input, axis=0)
    delta = input_x - baseline_x
    path_inputs = baseline_x + alphas_x * delta

    return path_inputs

def compute_gradients(model, path_inputs):
    """
    compute dependency of each field on whole result, compared to interpolated 'images'

    :param model: trained model
    :param path_inputs: interpolated tensors, shape: (21, 500, 21)
    :return: shape: (21, 500, 21)
    """
    with tf.GradientTape() as tape:
        tape.watch(path_inputs)
        predictions = model(path_inputs)

        outputs = []
        for envelope in predictions:
            outputs.append(envelope[0])
        outputs = tf.convert_to_tensor(outputs, dtype=tf.float32)

    gradients = tape.gradient(outputs, path_inputs)
    return gradients

def integral_approximation(gradients, method='riemann_trapezoidal'):
    """Compute numerical approximation of integral from gradients.
    Args:
    gradients(Tensor): A 4D tensor of floats with the shape
    (m_steps, img_height, img_width, 3).
    method(str): A string representing the integral approximation method. The
    following methods are implemented:
    - riemann_trapezoidal(default)
    - riemann_left
    - riemann_midpoint
    - riemann_right
    Returns:
    integrated_gradients(Tensor): A 3D tensor of floats with the shape
    (img_height, img_width, 3).
    """
    if method == 'riemann_trapezoidal':
        grads = (gradients[:-1] + gradients[1:]) / tf.constant(2.0)
    elif method == 'riemann_left':
        grads = gradients
    elif method == 'riemann_midpoint':
        grads = gradients
    elif method == 'riemann_right':
        grads = gradients
    else:
        raise AssertionError("Provided Riemann approximation method is not valid.")

    # Average integration approximation.
    integrated_gradients = tf.math.reduce_mean(grads, axis=0)

    return integrated_gradients

def integrated_gradients(model, baseline, input, m_steps=50, method='riemann_trapezoidal',
                         batch_size=32):
    """
    Args:
      model(keras.Model): A trained model to generate predictions and inspect.
      baseline(Tensor): 2D, shape: (500, 21)
      input(Tensor): preprocessed sample, shape: (500, 21)
      m_steps(Tensor): A 0D tensor of an integer corresponding to the number of
        linear interpolation steps for computing an approximate integral.
      method(str): A string representing the integral approximation method. The
        following methods are implemented:
        - riemann_trapezoidal(default)
        - riemann_left
        - riemann_midpoint
        - riemann_right
      batch_size(Tensor): A 0D tensor of an integer corresponding to a batch
        size for alpha to scale computation and prevent OOM errors. Note: needs to
        be tf.int64 and shoud be < m_steps. Default value is 32.
    Returns:
      integrated_gradients(Tensor): A 2D tensor of floats with the same
        shape as the input tensor.
    """

    # 1. Generate alphas.
    alphas = generate_alphas(m_steps=m_steps,
                             method=method)

    # Initialize TensorArray outside loop to collect gradients. Note: this data structure
    gradient_batches = tf.TensorArray(tf.float32, size=m_steps + 1)

    # Iterate alphas range and batch computation for speed, memory efficiency, and scaling to larger m_steps.
    for alpha in tf.range(0, len(alphas), batch_size):
        from_ = alpha
        to = tf.minimum(from_ + batch_size, len(alphas))
        alpha_batch = alphas[from_:to]

        # 2. Generate interpolated inputs between baseline and input.
        interpolated_path_input_batch = generate_path_inputs(baseline=baseline,
                                                             input=input,
                                                             alphas=alpha_batch)

        # 3. Compute gradients between model outputs and interpolated inputs.
        gradient_batch = compute_gradients(model=model,
                                           path_inputs=interpolated_path_input_batch)

        # Write batch indices and gradients to TensorArray.
        gradient_batches = gradient_batches.scatter(tf.range(from_, to), gradient_batch)

    # Stack path gradients together row-wise into single tensor.
    total_gradients = gradient_batches.stack()

    # 4. Integral approximation through averaging gradients.
    avg_gradients = integral_approximation(gradients=total_gradients,
                                           method=method)

    # 5. Scale integrated gradients with respect to input.
    integrated_gradients = (input - baseline) * avg_gradients

    return integrated_gradients

def choose_validation_points(integrated_gradients):
    """
    Args:
          integrated_gradients(Tensor): A 2D tensor of floats with shape (500, 21).
    Return: List of attributes for highlighting protein sequence
    """
    attr = np.zeros(500)
    for i in range(500):
        for j in range(21):
            if integrated_gradients[i][j].numpy() == 0:
                continue
            attr[i] = integrated_gradients[i][j].numpy()
    return attr

def visualize_token_attrs(sequence, attrs):
    """
    Visualize attributions for given set of tokens.
    Args:
    - tokens: An array of tokens
    - attrs: An array of attributions, of same size as 'tokens',
      with attrs[i] being the attribution to tokens[i]

    Returns:
    - visualization: HTML text with colorful representation of protein sequence
        build on model prediction
    """

    def get_color(attr):
        if attr > 0:
            red = int(128 * attr) + 127
            green = 128 - int(64 * attr)
            blue = 128 - int(64 * attr)
        else:
            red = 128 + int(64 * attr)
            green = 128 + int(64 * attr)
            blue = int(-128 * attr) + 127

        return red, green, blue

    # normalize attributions for visualization.
    bound = max(abs(max(attrs)), abs(min(attrs)))
    attrs = attrs / bound
    html_text = ""
    for i, tok in enumerate(sequence):
        r, g, b = get_color(attrs[i])
        if abs(attrs[i]) > 0.5:
          html_text += " <span style='color:rgb(%d,%d,%d);font-weight:bold'>%s</span>" % (r, g, b, tok)
        else: 
          html_text += " <span style='color:rgb(%d,%d,%d)'>%s</span>" % (r, g, b, tok)

    return html_text

## Input your sequence

In [6]:
seq = ''

adjust the sequence to be 500 amino acids long

In [7]:
if len(seq) > 500:
    seq = seq[:500]
else:
    seq = seq + (500 - len(seq))*'X'
    
print(seq)

MKFIVKTQRGMESVAANYIREAISDASVWASPMGYSGLIIVETSDENAGEKILEIPEVERLIPVIAEVPAELEAIVTTAEKIAPLISENETFAVKTKRRGKHGFTSMDVNRELGARIRELTNADVNLSWPDRVVQVEIIGDKAYISVVPGEEFRKFTPDKIDARKLFRKVTLVQMPYWGNHKACRSFGEKIGRAAQAFEVKELIIAPKEKMDAFELAEFIKGVKIGQESRHQIQREAYPWKVEKVPVSVWDLYQVVRDKRRGKRLLIITDPKGPTLAEVKDRLAKDMFYAKEVVIFVGSREGIPRGLFRFADYVVDLAPYMTFATEHGIPAALVSLWEVYEEYARKREKKTXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


-------------------------------------------------------------------------------------
### or try out one of the prepared sequences:

Positive sequence example

In [11]:
seq = 'MAQQEWLFGLHALQAVLEREPERILELMVLKGRTDERLGEIINQARRFGVSVQFCHRKVLDEKVGGSQHQGVVARAKPARALDENDLAVIVERADKPFLLVLDGVTDPHNLGACLRTADAAGVDAVIVPKDKSASLSGTVSKVACGAAETMPLVQVTNLARTLKTLQEAGVWIIGTAGETTQTLYDTRLDGPMALVMGAEGKGMRRLTRETCDELVKLPMAGSVSSLNVSVATGVCLYEVVRQRQQXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Negative sequence example

In [ ]:
seq = 'MAYTVVWFKRDLRVHDHAPLAHAAAHGPVLCLYVLEPSLWAQPDSALQHYQFLRESLRDLAQQLRSCGARLQLAVGETPEVLARLYALQPFARLVSHEETGNGATYARDLAVARWCRRQGVAWQEWPQHGVVRRLPSREQWHGLWQAHMQAPCLPPPLPASLRSVCLPWRDTWPAPETMGLSDAHDPPQRQRGGRAPGQQVLHDFLHARAAFYRGGISSPLTAPTACSRLSPYLALGCLGMREAVQATQQRQVQLKLQDAQQAAHVRQAFLAQFGQHGGGGDVGVPLRTALMRRVREDGRRYAVNLVIGQRVIAAQRGGAGSETGCELHGLSPDEKGLLFKPHTGFLDSEPSLSAVLCGWHEEPGWPCCHRLSXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

-------------------------------------------------------------------------------------

encode the sequence

In [12]:
seq_onehot = tf.one_hot([nucleo_dic[c] for c in seq], depth=21)

seq_onehot.shape

TensorShape([500, 21])

run integrated gradients and their visualization

In [13]:
baseline = tf.zeros(shape=(500, 21))

ig_attribution = integrated_gradients(model, baseline, seq_onehot, m_steps=200)
attrs = choose_validation_points(ig_attribution)

visualisation = visualize_token_attrs(seq, attrs)
HTML(visualisation)

model score for the input sequence:

In [14]:
print(model.predict(np.expand_dims(seq_onehot, axis=0), verbose=0))

[[0.99997294]]


### **Color explanation:**
**Red** amino acids influences the model to think the protein is knotted

**Blue** amino acids influences the model to think the protein is unknotted